In [ ]:
#!pip install Flask
#!pip install pymessenger
#!pip install https://github.com/oroszgy/spacy-hungarian-models/releases/download/hu_core_ud_lg-0.3.1/hu_core_ud_lg-0.3.1-py3-none-any.whl
#!pip install spacy==2.1
#!pip install unidecode

#A futtatáshoz kell hostolni és az url-t webhook-ként a Facebook app-hez kötni.
#Ehhez szólni kéne, de mint utólag kiderült, kb. az én facembe kell majd belépni, mert hitelesítési folyamaton kell
#végigmennie az appnek, mielőtt élesedhetne mindenkinek, ide pedig videók stb. minden kell :/

import unidecode
import random
from flask import Flask, request
from pymessenger.bot import Bot
import spacy
import hu_core_ud_lg
nlp = hu_core_ud_lg.load()

app = Flask(__name__)
ACCESS_TOKEN = 'EAAnMFMZAxZAUMBAC1s22arTMg4LObsBGuZCMItXgbZBu78cCZApnSAWbHuhgLzRUYZAB0hIxYYKZAr5GjRWK5jOfTVfvQjZCPxaRVIAzuX56ZC6CJD6Js3uHbFttmPoQpchaHfilZAclWIna6VoFRagRoYxF7zoyyvDTNmsI7Eov0QugZDZD'
VERIFY_TOKEN = 'VERIFY_TOKEN'
bot = Bot(ACCESS_TOKEN)

def verify_fb_token(token_sent): #a beérkező token egyezését ellenőrző függvény. Ha megegyezik, lefut, ha nem, errort dob ki
    if token_sent == VERIFY_TOKEN:
        return request.args.get("hub.challenge")
    else:
        return 'Invalid verification token'

#uses PyMessenger to send response to user
def send_message(recipient_id, response):
    bot.send_text_message(recipient_id, response) #pymessenger uzenetkuldo kodja
    return "success"
    

#requesteket befogadó link (endpoint) elkészítése (ide kerülnek az üzenetek)
@app.route("/", methods=['GET', 'POST'])
def receive_message():
    if request.method == 'GET': #A GET az ellenőrzési token beérkezésének interakcióját jelenti
        token_sent = request.args.get("hub.verify_token") #Ez gyakorlatilag egy biztonsági kód, ami megakadályozza a külső (nem FB) requesteket
        return verify_fb_token(token_sent)                #Előre meg kell adni a FB-nak, ezt visszaküldi a request esetén, ha nem matchel, nem fut le az app

    
    else: #Ha nem GET, akkor személy által küldött üzenet, kép stb.
        output = request.get_json()
        for event in output['entry']:
            messaging = event['messaging'] #leszűri a beérkező interakciók vagy "eventek" közül az üzenetekre
            for message in messaging:
                if message.get('message'):

                    recipient_id = message['sender']['id'] #Tudnunk kell, hogy ki küldte (azonosító): neki fogjuk vissza is küldeni

                    uzenetek = []
                    #Kapott üzenet lekérése
                    kapott_uzenet = message['message'].get('text')

                    kapott_uzenet_nlp = nlp(kapott_uzenet)

                    eredeti_szavak = []
                    lemma_szavak = []
                    ekezet_nelkuli_szavak = []

                    #szavak lemmatizálása
                    for szo in kapott_uzenet_nlp:
                        eredeti_szavak.append(szo.text)
                        lemma_szavak.append(szo.lemma_)

                    #Ékezetek eltávolítása
                    for i in lemma_szavak:
                        ekezet_nelkuli_szavak.append(unidecode.unidecode(i))

                    #Kisbetűvé alakítás
                    uzenet_kulcsszavak = [szo.lower() for szo in ekezet_nelkuli_szavak]

                    for x in uzenet_kulcsszavak:
                        a = " ".join(uzenet_kulcsszavak)
                     
                    if "szia" in uzenet_kulcsszavak or "hello" in uzenet_kulcsszavak or "szeva" in uzenet_kulcsszavak or "hali" in uzenet_kulcsszavak or "csa" in uzenet_kulcsszavak or "cso" in uzenet_kulcsszavak or "szio" in uzenet_kulcsszavak or "udv" in uzenet_kulcsszavak:
                        koszonesek = ["Szia!", "Üdvözlet!", "Hello!"]
                        valasz = random.choice(koszonesek)
                        send_message(recipient_id,valasz)
                                             
                    
                    #Hivatkozasok
                    if "hivatkozas" in uzenet_kulcsszavak or "hivatkozik" in uzenet_kulcsszavak or "hivatkozzam" in uzenet_kulcsszavak or "szovegkoz" in uzenet_kulcsszavak or "lehivatkozni" in uzenet_kulcsszavak or "hivatkozz" in uzenet_kulcsszavak or "kereszthivatkozas" in uzenet_kulcsszavak or "kereszt" in uzenet_kulcsszavak or "kereszthivatkozik" in uzenet_kulcsszavak or "forrasjegyzek" in uzenet_kulcsszavak or "irodalomjegyzek" in uzenet_kulcsszavak or "hivatkozasjegyzek" in uzenet_kulcsszavak:
                    
                        #Több szerző esetén
                        if "ket" in uzenet_kulcsszavak or "harom" in uzenet_kulcsszavak or "tobb" in uzenet_kulcsszavak or "szerzok" in uzenet_kulcsszavak or "2" in uzenet_kulcsszavak or "3" in uzenet_kulcsszavak or "sok" in uzenet_kulcsszavak:
                            valasz = "Egy- illetve két szerző esetében a szövegközben a vezetéknevek feltüntetésével (Németh, 2002) illetve (Németh – Zsótér, 2007), három vagy több szerző esetén az első szerző megnevezése után et al. (et alii latin kifejezés jelentése: és mások) kifejezéssel rövidíthető a hivatkozás (Németh et al., 2002). Az et al. használata csak szövegközben megengedett, a hivatkozásjegyzékbe minden nevet ki kell írni!"
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)
                        
                            
                            
                        #Tippek, trükkök, tanácsok
                        if "tanacs" in uzenet_kulcsszavak or "tipp" in uzenet_kulcsszavak or "trukk" in uzenet_kulcsszavak or "eligazitas" in uzenet_kulcsszavak:
                            valasz = """Nagyon figyeljünk a helyes és pontos hivatkozásokra, hisz ez 1) komoly hatással van a dolgozat külalakjára, 2) biztosítja érvelésünk visszakereshető alátámasztását, és 3) más gondolatait nem tüntetjük fel a sajátunkként. Utóbbi tekintetben a hivatkozás elhagyása plágium vétségét jelenti, ami az egyetemen komoly szankciókat von maga után. Bár kétségkívül az a legsúlyosabb kihágás, ha az egész írás más mű(vek) ki- vagy összeollózott másolata, a plágium ennél szélesebb kategória. Ugyanúgy vonatkozik egyes gondolatokra, eredményekre – akár máshogy megfogalmazva is. Gondos hivatkozással a legkisebb gyanú is elkerülhető!
                            
                            Előzetes kutatásunk során még nem tudhatjuk, mely műveket fogjuk felhasználni, illetve melyek nélkülözhetők. Érdemes folyamatosan egy minden szükséges bibliográfiai adatot tartalmazó forrásjegyzéket vezetni az összes kézbe vett írásról! Így elkerülhetjük az utolsó pillanatbeli kétségbeesett és időrabló kutakodást egy végül mégis fontosnak bizonyuló forrás adatai után, illetve megelőzhetjük a plagizálás veszélyét.
                            
                            Ellenőrizzük a forrásainkat! Csak megbízható és tudományos értékű művekre támaszkodjunk, hiszen a forrásaink nagy részben meghatározhatják az egész írás minőségét is.
                            
                            Ha egy mód van rá, közvetett hivatkozás helyett keressük meg az eredeti forrást, így biztosak lehetünk tartalmában. 
                            
                            Google Scholar-ról könnyen kimásolható az adott mű hivatkozása az idézőjel gombra kattintva (APA), azonban ezeket érdemes átnézni, hogy az minden szükséges adatot tartalmazzon.

                            """
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)

                            
                        #Kereszthivatkozás
                        elif "kereszt" in uzenet_kulcsszavak or "kereszthivatkozas" in uzenet_kulcsszavak or "kereszthivatkozik" in uzenet_kulcsszavak:
                            valasz = """Közvetett hivatkozás, avagy hivatkozás a hivatkozás hivatkozására, ha a dolgozatban hivatkozott mű a hivatkozott résznél egy másik műre hivatkozik. Tehát amikor a dolgozatban idézett Németh műve a hivatkozott résznél a dolgozatban nem hivatkozott Zsótér művét idézi.

                            Formáját tekintve több jó megoldási forma van, de egyértelműnek kell lennie, hogy ki idéz kit. A pontosság érdekében kereszthivatkozásnál feltüntethető mindkét mű keletkezési ideje.

                            Példa:

                            szövegközben: Zsótért idézi Németh, 2006, 32. old. VAGY (Zsótér, 2002-t idézi Németh, 2006, 32. old.)
                            hivatkozásjegyzékben: Németh Ders (2006): Demonstrátori visszaemlékezéseim. Budapest, Aula Kiadó

                            Tehát itt már Zsótér művét nem kell említeni. Itt jó viszonyítási alapként szolgálhat az a szabály, hogy olyan könyvet/cikket nem szabad forrásjegyzékbe betenni, amit nem is fogtunk a kezünkbe (ezért nem szabad közvetlenül az idézett forrásra hivatkozni, még ha annak pontos hivatkozása benne is van az olvasott műben). Ennek gyakorlati jelentősége is lehet: előfordulhat, hogy az általunk olvasott mű félreértelmezi, rosszul interpretálja az eredetit, a kereszthivatkozással ezt a “felelősséget” kerülhetjük el.
                            Remélem érthető volt!
                            """
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)
                       
                            
                        #Hivatkozásjegyzék
                        elif "hivatkozasjegyzek" in uzenet_kulcsszavak or "forrasjegyzek" in uzenet_kulcsszavak or "irodalomjegyzek" in uzenet_kulcsszavak:
                            valasz = """Ezt szokták forrásjegyzéknek, hivatkozásjegyzéknek vagy egyszerűen csak hivatkozásoknak nevezni. Feladata, hogy a szövegközi hivatkozásokat a dolgozat végén egy listába gyűjtse. Ami ebben szerepel, kell hogy szerepeljen a szövegközi hivatkozások között is, és ez fordítva is igaz. 

                            Egyes tudományos dolgozatokban a hivatkozás- és az irodalomjegyzéket külön tüntetik fel. A hivatkozásjegyzék e logika szerint a szövegközi hivatkozások jegyzéke, míg az irodalomjegyzék minden olyan forrást tartalmaz, mely a dolgozat megírása során a szerzőt inspirálta, de közvetlen gondolatot nem használt fel belőle.

                            A hivatkozásjegyzék formai összeállításánál fontos alapelv a következetesség, azaz az egyes tételek formázása végig következetes legyen.  Az irodalom jegyzékben, a könnyebb visszakereshetőség érdekében, az egyes tételek a szerző vezeték neve szerinti ABC sorrend követik egymást. 

                            A különböző dokumentumok megjelenése a hivatkozásjegyzékben is eltérő lehet, ezek jellemző példáit a következő fejezetben találhatjátok.
                            """
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)

                            
                        #Szó szerinti hivatkozás   
                        elif "szoszerinti" in uzenet_kulcsszavak or "szoszerint" in uzenet_kulcsszavak or "idezet" in uzenet_kulcsszavak or "idezek" in uzenet_kulcsszavak or "szerint" in uzenet_kulcsszavak:
                            valasz = """Minden esetben idézőjelbe kell tenni az idézett szöveget és oldalszámot is jelölni kell! Pl.: (“Minden esetben idézőjelbe kell tenni...” (Ábrahám-Fekete, 2012, 2. old.)).
                                    Egy rosszul kivitelezett szó szerinti hivatkozás könnyen plágiumhoz vezethet, úgyhogy különösen figyelj!"""
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)
                       

                            
                        #Oldalszám
                        elif "oldalszam" in uzenet_kulcsszavak or "felsorolas" in uzenet_kulcsszavak or "forditas" in uzenet_kulcsszavak or "szama" in uzenet_kulcsszavak:
                            valasz = """Szó szerinti idézet, listaszerű felsorolás, fordítás, táblázat, számszerűsített adatközlés vagy ábra esetén a pontos oldalszám is kötelezően megjelölendő (Németh – Zsótér, 2007,77. old.).
                            Az oldalszám feltüntetésének számos módja van, itt is a következetesség a legfontosabb. Alkalmazható a magyar (27. old./oldal), angol (p. 27, több oldal esetében pp. 27-28), de más elfogadott megoldás is (2007:77, stb.).
                            Azon esetekben, amikor a szó szerinti idézetnek, ábrának-táblázatnak, vagy hivatkozott adatnak nincs meghatározható oldalszáma (például internetes honlapon szerepelt), akkor természetesen itt sem lehet feltüntetni. """
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)


                        #Táblázat és ábra
                        elif "tablazat" in uzenet_kulcsszavak or "abra" in uzenet_kulcsszavak or "tablaz" in uzenet_kulcsszavak:
                            valasz = """Sokszor ezek a vizualizációk segítenek szemléletesebben, egyszerűbben közölni az információt, ilyenkor használatuk ajánlott. Már csak azon okból kifolyólag is, mert megtörik a folytonos szöveg monotóniáját, olvasmányosabbá teszik a dolgozatot. Fontos észben tartani, hogy hivatkozásuk módjára speciális szabályok érvényesek: minden esetben közvetlenül az adott objektum (illetve annak címmegjelölése) alatt fel kell tüntetni a forrást, pontos oldalszámmal. Az alábbi esetek fordulhatnak elő:

                            1) Ha saját készítésű objektumról van szó (pl. egy koncepcionális ábra esetén), erről is informálni kell az olvasót – amennyiben ez nem teljesen egyértelmű. Ezt például (1. ábra - XY, forrás: saját koncepció), vagy más módon szoktuk jelezni.

                            2) Más szerző ábrájának, stb. átvétele estén minden esetben fel kell tüntetni a pontos oldalszámot. A változtatás nélkül átvett ábrák, grafikonok esetében értelemszerűen az általánosnak megfelelően, oldalszámmal kiegészítve (például: 2. ábra - YZ, forrás: Németh-Zsótér, 2007, 3. oldal).

                            3) Ha mégis változtatunk az eredeti objektumon, akkor ugyancsak fel kell tüntetni a forrást, a végére odaírva, hogy „alapján” vagy más utalás a változtatásra. Előfordulhat, hogy pl. több táblázatot gyúrunk egybe, ekkor az összes forrást fel kell sorolni (például: 2. ábra - YZ, forrás: Németh-Zsótér, 2007, 3. oldal alapján saját elképzelés).

                            4) Ha közvetett hivatkozásról (lásd korábban) van szó, akkor a szövegközi megoldáshoz hasonlóan járunk el: feltüntetjük az eredeti forrást, idézi: ahol mi láttuk az ábrát, és a hivatkozásjegyzékben csak ez utóbbi szerepel."""
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)

                            
                        #Adatpont
                        elif "adat" in uzenet_kulcsszavak or "adatpont" in uzenet_kulcsszavak:
                            valasz = """Adat hivatkozások esetében a közvetlenül a felhasznált adatot követő (!) hivatkozásban a szerzőt és évszámot követően ugyancsak fel kell tüntetni a pontos oldalszámot.

                            Szövegközben: (Schwab, 2017, 9. old.)

                            Hivatkozásjegyzékben: Schwab, Klaus (2017): The Global Competitiveness Report 2017-2018. World Economic Forum"""
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)
                            
                            
                            
                        #Hiányzó adatok    
                        elif "hianyzo" in uzenet_kulcsszavak or "hianyos" in uzenet_kulcsszavak or "talalhato" in uzenet_kulcsszavak or "nincs" in uzenet_kulcsszavak:
                            valasz = """Sokszor előfordulhat – elsősorban internetes cikkeknél, de könyvek esetén is van rá példa –, hogy az adott forrásról nem áll rendelkezésre az összes szükséges adat. Ezt jelölni kell a hivatkozásban is:

                            Hiányzó évszám: évszám helyett az é.n. (év nélkül), n.d. (nincs dátum) vagy s.a. (sine annum – latinul jelenti ugyanazt) rövidítés szerepel (Németh Zsótér, s.a.)

                            Hiányzó szerző: Amennyiben tényleg kideríthetetlen, hogy kitől származik az offline mű, akkor a következő lehetőség adódik: a címre hivatkozunk (vagy annak első pár szavára hosszabb cím esetén) …-tal jelölve a szövegközben, a hivatkozásjegyzékben szerző helyett a rövid szövegközi formát használva, a későbbiekben leírt szabvány módon. (Offline esetben, internetes forrás esetén lásd: III.2)

                            Példa

                            szövegközben: (Problémáim…, 2006)
                            hivatkozásjegyzékben: Problémáim… (2006): Problémáim a vállalatgazdaságtannal, Közgazdász, 2006/3. szám, 3. old.
                            """
                            send_message(recipient_id,valasz)
                            send_message(recipient_id,a)
                                                        
                            
                        #Könyv
                        elif "konyv" in uzenet_kulcsszavak or "kiado" in uzenet_kulcsszavak:
                            valasz_1 = """A) Könyvek 
                            Könyvek szövegközi hivatkozása esetén a szerző az évszám - speciális esetekben oldalszám - feltűntetése szükséges.
                            Könyvek hivatkozásjegyzékben esetében a hivatkozás jegyzékben a következő adatelemeket kell megadni: szerző, cím-alcím, kiadás (abban az esetben, ha ez releváns), kiadó, kiadás helye, kiadás éve.

                            Példa:

                            szövegközben: (Chikán, 2005) 
                            hivatkozásjegyzékben: Chikán A. (2005): Vállalatgazdaságtan, 3. átdolgozott, bővített kiadás, AULA Kiadó, Budapest


                            Külföldi szerzők esetében szövegközben ugyancsak vezetéknévvel, hivatkozásjegyzékben pedig Vezetéknév, Keresztnév formátumban hivatkozzunk.

                            Példa:

                            szövegközben: (Williamson, 1985) 
                            hivatkozásjegyzékben: Williamson, Oliver E. (1985): The Economic Institutions of Capitalism, The Free Press, New York"""

                            valasz_2 = """B) Könyv fejezet

                            Könyv fejezetére történő hivatkozás esetén meg kell adni a kötet teljes bibliográfiai tételét az In: megjegyzés után (-ban, -ben értelemben). Ez akkor szükséges, ha a könyv egy többszerzős tanulmánykötet vagy szöveggyűjtemény. (A könyvtár adatbázisában például a „könyv analitika” típusú kötetek, felismerhetőek még onnan, hogy szerzőként a szerkesztő van feltüntetve – utóbbit a hivatkozásban is jelölni kell egy zárójeles (szerk.) ill. (ed.) kitétellel.)

                            Példa: 

                            szövegközben: (Németh-Zsótér, 2007)
                            hivatkozásjegyzékben: Németh Ders – Zsótér Csaba (2007): Demonstrátor voltam. In: Gáspár Judit (szerk.) (2007): Demonstrátori visszaemlékezések. Budapest, Aula Kiadó, 496-506 old."""

                            valasz_3 = """Tehát, ha egy „normál” – akár többszerzős! – könyv egy fejezetére hivatkozunk, akkor csupán az adott rész oldaltartományát kell megadni a szövegtörzsben (Németh – Zsótér, 2007, 50-76. old), a hivatkozásjegyzékben már a teljes mű szerepel.

                            Hüvelykujj szabályként szolgálhat, hogy akkor használjuk az In: formát, ha a köteten belül, az egyes részek elején külön feltüntetik a szerzőt és címet, nem csupán a fejezetcím szerepel."""

                            
                            send_message(recipient_id, valasz_1)
                            send_message(recipient_id,valasz_2)
                            send_message(recipient_id,valasz_3)
                            send_message(recipient_id,a)
                                              
                            
                        #Kiadványok
                        elif "kiadvany" in uzenet_kulcsszavak or "rajk" in uzenet_kulcsszavak:
                            valasz = """Hasonlóan kell kezelni a rajkos kiadványokat. Amennyiben az adott tanulmány egy összefoglaló egy eredeti műből, vagy a szerzőétől eltérő gondolat jelenik meg benne, úgy szövegközben kereszthivatkozni kell, míg a hivatkozásjegyzékben a fent említett módon kell eljárni.

                                    Példa: 

                                    szövegközben: (Biro ., 2015 idézi Jackson, 2012-t, pp. 52-56)
                                    hivatkozásjegyzékben: Biro. (2015): Összefoglaló a hálózatokról. In: Borza (szerk.) (2015): A hálózatok csapdájában. Budapest, Alinea Kiadó, pp. 12-18.

                                    """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                            
                            
                        #Folyóiratban lévő tanulmány
                        elif "folyoirat" in uzenet_kulcsszavak or "paper" in uzenet_kulcsszavak or "tanulmany" in uzenet_kulcsszavak:
                            valasz = """Szövegközben a korábbihoz hasonlóan hivatkozunk, míg a hivatkozásjegyzékben feltüntetendő adatok (egy javasolt sorrendben): Név (vezetéknév, keresztnév – külföldi forrásnál a korábbihoz hasonlóan!), Kiadás éve, Cím – alcím, Folyóirat, Évfolyam, Sorszám, Oldalszám (külföldi publikáció esetében a folyóirat például Volume (Vol.), a sorszám Issue vagy No. jelöléssel tüntethető fel.

                            Példa:

                            szövegközben: Bakonyi (2010)
                            hivatkozásjegyzékben: Bakonyi Z. (2010). Az oksági homályosság relatív megközelítése. Vezetéstudomány, 41. évfolyam, 6. szám, 40-49. oldal.
                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                            
                            
                        #Internetes szöveg, honlap hivatkozása
                        elif "cikk" in uzenet_kulcsszavak or "weblap" in uzenet_kulcsszavak or "honlap" in uzenet_kulcsszavak or "internet" in uzenet_kulcsszavak or "net" in uzenet_kulcsszavak: 
                            valasz_1 = """Elektronikus (internetes) forrásoknál meg kell adni a szerző nevét és a munka címét, a keletkezés évét (amennyiben fel van tüntetve), a forrás teljes elérhetőségét, valamint az időpontot, amikor a forrás letöltöttük. A forrás pontos URL-jének és a letöltés időpontjának jelentősége az elektronikus tartalmak gyors keletkezéséből és eltűnéséből fakad. Ezért egyébként nem árt a megtalált tartalmakat rögtön le is tölteni! Mostanában sok folyóirat megjelenik elektronikus formában, ha a hivatkozott cikket elektronikus formában olvastuk, azt is közölni kell.

                            A) Interneten olvasott folyóirat-cikkek 
                            Ezek esetében az offline folyóiratokhoz hasonlóan kell eljárni, de a hivatkozásjegyzékben fel kell tüntetni a letöltés helyét és idejét is.

                            Példa:

                            szövegközben: (Németh-Zsótér, 2007)
                            hivatkozásjegyzékben: Németh D. – Zsótér Cs. (2007): Demonstrátor voltam. letöltés helye: http://www.vallgazd.hu/demonstrator/nemeth_zsoter.html; letöltés ideje: 2007. március 14.

                            Honlapon talált egyéb források esetében a korábban leírtakhoz képest az alábbiakra kell odafigyelni."""

                            valasz_2 = """ A1) Szerző kérdése

                            1, Amennyiben egy online írás egyértelműen köthető egy vagy több szerzőhöz, úgy mindenképpen a szerző nevével kell feltüntetni a korábban leírtak szerint.

                            Példa:

                            szövegközben: (Ábrahám-Fekete, 2012)
                            hivatkozásjegyzékben: Ábrahám Zs. – Fekete T. (2012): Kötött fogású hivatkozási gyakorlatok a Rajk László Szakkollégiumban, publicisztika, letöltés helye: http://www.nso.hu/1239621, letöltés ideje: 2012. április 21."""

                            valasz_3 = """Amennyiben a szerző neve nem ismert, úgy a honlap, vagy a honlap tulajdonosa tekinthető szerzőnek, a többi adatot a korábban leírtak szerint kell feltüntetni.

                            Példa:

                            szövegközben: (nso.hu, 2012) VAGY (Nemzeti Sport honlapja, 2012), stb…
                            hivatkozásjegyzékben: nso.hu (2012): Kötött fogású hivatkozási gyakorlatok a Rajk László Szakkollégiumban, publicisztika, letöltés helye: http://www.nso.hu/1239621, letöltés ideje: 2012. április 21.
                            VAGY: Nemzeti Sport honlapja (2012): Kötött fogású hivatkozási gyakorlatok a Rajk László Szakkollégiumban, publicisztika, letöltés helye: http://www.nso.hu/1239621, letöltés ideje: 2012. április 21.

                            3, Amennyiben internetes forrásból hivatkozunk adatokat, nem mindig áll rendelkezésre oldalszám, ilyen esetben – ahogy azt már jeleztük – nem tudjuk feltüntetni. Amennyiben az adatok forrása egy változó URL-ű internetes adatbázis, úgy a honlapot kell szerzőként feltüntetni.

                            Példa:

                            szövegközben: (OECD, 2012)
                            hivatkozásjegyzékben: OECD (2012): Gross domestic product (GDP), letöltés helye: http://www.oecd.org, letöltés ideje: 2012. május 1."""


                            valasz_4 = """A2) Cím használata

                            Amennyiben az írás, stb. nem rendelkezik egyértelmű címmel, úgy törekedjünk arra, hogy a megnevezéssel egyértelműsítsük a forrás azonosítását (lásd: az OECD-s példa)

                            Példa:

                            szövegközben: (OECD, 2012)
                            hivatkozásjegyzékben: OECD (2012): Az OECD adatbázisa, letöltés helye: http://www.oecd.org, letöltés ideje: 2012. május 1.


                            A2) Évszám pontos használata

                            Az évszám esetében oda kell figyelni arra, hogy a letöltés ideje nem feltétlenül esik egybe a forrás keletkezésének idejével! Az elektronikus oldalak esetében általában feltüntetésre kerül a publikálás évszáma (elektronikus folyóiratok esetében a cikk keletkezése, stb.), ilyenkor évszámként ez alkalmazandó. Amennyiben ez nem áll rendelkezésre, úgy ajánlott a hiányzó évszám (s. a.) korábban bemutatott alkalmazása."""

            
                            send_message(recipient_id, valasz_1)
                            send_message(recipient_id, valasz_2)
                            send_message(recipient_id, valasz_3)
                            send_message(recipient_id, valasz_4)
                            send_message(recipient_id,a)
                              
                        
                        #Interjú
                        elif "interju" in uzenet_kulcsszavak or "interjuzik" in uzenet_kulcsszavak or "interjualany" in uzenet_kulcsszavak or "interjuztam" in uzenet_kulcsszavak or "forras" in uzenet_kulcsszavak:
                            valasz = """Ez egy igen bevett kvalitatív kutatási forma és szeretjük magunk végezni, hogy a munkánkat legjobban segítő kérdések mentén járjunk végig egy témakört. Mégis előfordulhat, hogy nincs lehetőségünk primer kutatást végezni, vagy kiegészítésként vonunk be mások által készített interjúkat is a dolgozatba.  Világosnak kell lennie, hogy kinek a gondolatait is idézzük, szövegközben tehát az eredeti interjúalanyt tüntetjük fel az interjú készítésének idejével együtt. Az interjúztató személyét, megjelenésének pontos elérését, időpontját és helyszínét a hivatkozásjegyzékben tüntetjük fel. 
                            Interjúk esetében ezt szövegközben és hivatkozásjegyzékben is fel kell tüntetni, illetve lehetőleg a dolgozat végére – mellékletként – csatolni kell az interjú kivonatát, de nem kell legépelni az interjút.

                            Szövegközben: (Christensen,2014)
                            Hivatkozásjegyzékben Christensen, Clayton (2014) – A kapitalista dilemmája (Szalai Bálint interjúja), 2014.11.25. index.hu, Rossz úton jár a kapitalizmus, Elérhető: http://index.hu/gazdasag/2014/11/25/christensen_interju/, Letöltve: 2016. 10. 25., Budapest
                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
            
                              
                        #Törvények
                        elif "torveny" in uzenet_kulcsszavak or "cikkej" in uzenet_kulcsszavak or "paragrafus" in uzenet_kulcsszavak or "rendelet" in uzenet_kulcsszavak or "forras" in uzenet_kulcsszavak:
                            valasz = """
                            Törvények esetében a szöveg közben egyértelműen utalnunk kell rá, hogy melyik törvényre hivatkozunk (neve, évszáma, sorszáma), a hivatkozásjegyzékben pedig – például – a többi hivatkozástól elkülönítve felsorolhatjuk a felhasznált jogi háttér elemeit.

                            Példa:

                            szövegközben: a 2006-os, ún. gyógyszer-gazdaságossági törvény szerint
                            hivatkozásjegyzékben: Gyógyszer-gazdaságossági törvény: 2006. évi XCVIII. törvény, 8. paragrafus

                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                              
                        
                        #Internetes videó, hanganyag
                        elif "video" in uzenet_kulcsszavak or "hanganyag" in uzenet_kulcsszavak or "portre" in uzenet_kulcsszavak or "zene" in uzenet_kulcsszavak:
                            valasz = """Az elsődleges szempontok ez esetben is a beazonosíthatóság és a következetes hivatkozás.

                            Példa:

                            szövegközben: “nincs megállás ” (Queen Official, 2008, 0:31-0:33), ahogy Freddie is mondja
                            hivatkozásjegyzékben: Queen Official (2008): Don’t Stop Me Now, youtube.com, letöltés helye:https://www.youtube.com/watch?v=HgzGwKwLmgM&list=RDHgzGwKwLmgM&start_radio=1, letöltés ideje: 2018. május 5.

                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                              
                              
                        #Szóbeli közlés
                        elif "szobeli" in uzenet_kulcsszavak or "kozles" in uzenet_kulcsszavak or "mondas" in uzenet_kulcsszavak or "mond" in uzenet_kulcsszavak or "mondja" in uzenet_kulcsszavak in uzenet_kulcsszavak:
                            valasz = """A szóbeli közlés nem a legmegbízhatóbb formája a hivatkozásnak, de szükséges ha más módon nem tudjuk feltüntetni a gondolat eredetét. Az egyetemen elhangzott tantárgyi kereteken belüli előadások, meghívott előadók (pl. Simon díjas) is ebbe a kategóriába származhat. Figyelem! Csak akkor használjuk ezt, ha valóban semmilyen megbízhatóbb írásos jegyzet, diasor nem született az alkalomról. Szövegközben ez az előadó nevét és az előadás idejét jelenti, míg hivatkozásjegyzékben fel kell tüntetni az előadás pontos címét, a tantárgyat vagy programot, amely keretében meghallgattuk az előadást, a pontos dátumot illetve az előadás helyszínét.
 
                            TIPP: Beszédek esetén érdemes visszakeresni a beszéd leiratát, pl.: miniszterelnöki beszédek esetén. Érdemes ilyenkor azt hivatkozni. 

                            Példa:

                            szövegközben: (Christensen, 2014)
                            hivatkozásjegyzékben: Christensen, Clayton (2014) Romboló innováció, Simon- díjas ceremónia, 2014. 11.25, Magyar Tudományos Akadémia
                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                              
                        
                        #Gondoaltnál nagyobb egység hivatkozása egy anyagból, vagy néhány mű összefoglalásaként előálló fejezet
                        elif "osszefoglalo" in uzenet_kulcsszavak or "osszefoglal" in uzenet_kulcsszavak or "fejezet" in uzenet_kulcsszavak or "osszesites" in uzenet_kulcsszavak or "forras" in uzenet_kulcsszavak in uzenet_kulcsszavak:
                            valasz = """Ha hosszabb terjedelemben (egész oldal, adott fejezet(rész)) egy adott forrásból hivatkozik a szerző, akkor érdemes az elején lábjegyzetbe (vagy akár a szövegbe) beleírni: „Az alábbi fejezet megírásakor nagyban támaszkodtam Németh-Zsótér (2007) művére.” vagy egy hasonló tartalmú megjegyzést.
                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                              
                        #Vállalati beszámoló (mérleg, eredménykimutatás, kiegészítő melléklet)
                        elif "vallalat" in uzenet_kulcsszavak or "vallalati" in uzenet_kulcsszavak or "merleg" in uzenet_kulcsszavak or "eredmenykimutatas" in uzenet_kulcsszavak or "melleklet" in uzenet_kulcsszavak or "kieg" in uzenet_kulcsszavak or "CF" in uzenet_kulcsszavak or "cashflow" in uzenet_kulcsszavak:
                            valasz = """Alapvetően ezek az adatok online formátumban érhetők el az e-beszámoló, cégjegyzék, Opten vagy más céges adatbázisok internetes felületeiről. Ebből adódóan hivatkozásuk is hasonló a már korábban említett adatbázisokhoz. Bár adatokról beszélünk jogosan felmerülhet az oldalszám igénye, sok esetben ezek egyoldalas dokumentumok így külön kiemelés nem szükséges, azonban amennyiben lehetőségünk van érdemes minél pontosabb elérhetőséggel szolgálni és feltüntetni az oldalszámot.  Szövegközben szerepelnie kell az adatbázist szolgáltató internetes jegyzék nevének, majd az adat megjelenésének időpontjának. A hivatkozásjegyzékbe ez kiegészül az adott vállalat nevével, a céges kimutatás formájával.  
 
                            Példa:

                            szövegközben: (Opten, 2015)
                            hivatkozásjegyzékben: Opten (2015): Boldog Kft kiegészítő melléklete 10. o.

                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                              
                        
                        #Internetes szöveg, honlap hivatkozása
                        elif "dvd" in uzenet_kulcsszavak or "film" in uzenet_kulcsszavak:
                            valasz = """Sokszor nyerünk ihletet beszámolótémánkhoz, vagy annak egy-egy gyakorlati megjelenítésén a korábban látott filmek, videók hatására.  Szövegközben hasonlóan áll össze, mint egy könyv hivatkozásánál, ahol a szerző nevét felváltja a rendező szervezet vagy személy neve, majd a film,videó megjelenésének dátuma. 
                            Míg a hivatkozásjegyzékben ezek az adatok kiegészülnek a film címével, megjelenés helyével és a filmet forgalmazó válltalat nevével.  

                            Példa:

                            szövegközben: (Snyder, 2004)
                            hivatkozásjegyzékben: Snyder, Zack (2004) - Holtak hajnala, [DVD], USA: Universal Studios

                            """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                              
                        #Adatbazis hivatkozasa
                        elif "adatbazis" in uzenet_kulcsszavak:
                            send_message(recipient_id,"masodik belépés")
                            valasz_1 = """Az adatbázisok sok forrásból származhatnak és számos esetben ez a hivatkozás módjára is hathat. Az adatbázisok lehetnek idegen vagy saját szerkesztésűek. Sok esetben a honlap maga is jelzi a preferált hivatkozási formát, ebben az esetben az követendő.

                            A) Idegen szerkesztésű adatbázis

                            A mások által összeállított adatbázisnál szövegközben az adatbázis összeállítójának (szerzőjének) vezetéknevét - vagy honlap esetén a honlap nevét kell megadni, valamint az adatbázis legutolsó frissítésének idejét. A hivatkozásjegyzékben az 1) összeállító neve, 2) évszám, 3) adatbázis címe, 4) honlap neve (ha van), 5) honlap állandó URL-je valamit 6) a letöltés ideje a megfelelő forma. 

                            Példa:

                            szövegközben: (OECD, 2017)
                            hivatkozásjegyzékben: OECD (2017): Inflation (CPI), oecd.org, letöltés helye: https://data.oecd.org/price/inflation-cpi.htm, letöltés ideje: 2018. február 1."""

                            valasz_2 = """B) Saját szerkesztésű adatbázisok
                            
                            Abban az esetben, ha különböző adatforrásokból (adatbázisokból) saját magad állítasz össze egy adatbázist (értsd: összeraksz különböző adatokat, kiszámítasz új dolgokat) akkor célszerű egy rövid bekezdésben leírni az egyes adatrészek forrásait, azokat megfelelően hivatkozva (lásd: A fejezet). Ezek után a szövegben hivatkozhatsz rá úgy mint saját adatbázis pl.: ábrák, táblázatok alatt.

                            Abban az esetben, ha az adatokat online forrásból scrapelted, akkor egy rövid bekezdés keretében szintén írd le, hogy mely honlapok scrapeléséből jött létre az adatbázisod, majd szövegközben saját névvel és a scrapelés időpontjával hivatkozd le. Ilyenkor (saját adatbázis, 2018)-ként is elfogadható. Ilyen esetekben viszont mindig mellékeld az adatokat, pl.: Google Drive vagy Github segítségével, melynek elérhetőségét a hivatkozásjegyzékben ildomos elhelyezni.  A lényeg, hogy a link nyílt legyen. 

                            Példa:

                            szövegközben: Az adatokat a Kuruc Info online újságról scrapeltem le. Az adatbázist én állítottam össze (Kovács, 2017).
                            hivatkozásjegyzékben: Kovács B. (2017): A szélsőjobb hatása Magyarországra, saját adatbázis, elérhetőség: https://www.dropbox.com/sh/2ebfiuzau02ns6y/"""
                            send_message(recipient_id, valasz_1)
                            send_message(recipient_id, valasz_2)
                            send_message(recipient_id,a)
                                                                                                     

                        else:
                            valasz = """Bocsi, sajnos nem értem :( Légyszi használj ékezeteket és próbáld meg egyszerűbben, valamint ne feledkezz meg a "hivatkozás" kulcsszó valamilyen formájáról! Pl.: "Táblázat hivatkozás" vagy "Hogyan hivatkozzak táblázatot?" """
                            send_message(recipient_id, valasz)
                            send_message(recipient_id,a)
                                                                            
                              
                    else:
                        valasz = """Bocsi, sajnos nem értem :( Légyszi használj ékezeteket és próbáld meg egyszerűbben, valamint ne feledkezz meg a "hivatkozás" kulcsszó valamilyen formájáról! Pl.: "Táblázat hivatkozás" vagy "Hogyan hivatkozzak táblázatot?" """
                        send_message(recipient_id, valasz)
                        send_message(recipient_id,a)
                        
                    
                    uzenet_kulcsszavak = []
                        
    return "Message Processed"


if __name__ == "__main__":
    app.run()